In [57]:
import numpy as np
from scipy.sparse import (csr_matrix, csr_matrix, coo_matrix,
                bsr_matrix, dia_matrix, dok_matrix, lil_matrix)
from numpy.random import rand
from time import time
import random

n = 10000 # dimension of matrix

# List of Lists format (lil_matrix)

Create a list of list matrix that is of size n x n.  We will fill that one with random values.  Also create lil matrix that is identity.

In [33]:
lil1 = lil_matrix((n,n))
lilIdentity = lil_matrix((n,n))

Fill the first one hundred values of the zeroth row with random numbers over the distribution [0, 1).  Then fill the diagonal also with random numbers.

In [34]:
lil1[0, :100] = rand(100)
lil1.setdiag(rand(n))


Fill in the identity matrix.

In [35]:
for i in range(n):
    lilIdentity[i,i] = 1

See how long it takes to multiply a lil matrix with another lil matrix.

In [36]:
time1 = time()
lil1 = lil1*lilIdentity
print("Time(s) for lil multiply: " + str(time() - time1))

Time(s) for lil multiply: 0.004474163055419922


Convert them to dense matrices and see how long the multiply takes.

In [37]:
dense1 = lil1.toarray()
denseIdentity = lilIdentity.toarray()
time1 = time()
dense1 = dense1 * denseIdentity
print("Time(s) for dense multiple: " + str(time() - time1))

Time(s) for dense multiple: 1.6002421379089355


Now how does adding matrices compare?

In [38]:
lil2 = lil1
time1 = time()
lil2 = lil2 + lil2
print("Time(s) for lil add: " + str(time() - time1))

Time(s) for lil add: 0.0006101131439208984


In [39]:
dense1 = lil1.toarray()
denseIdentity = lilIdentity.toarray()
time1 = time()
dense1 = dense1 + dense1
print("Time(s) for dense add: " + str(time() - time1))

Time(s) for dense add: 0.6401793956756592


What happens when a lil matrix has many values?

In [40]:
lilManyValues = lil_matrix((n,n))
numValues = int(n*n / 10)
for iter in range(numValues):
    i = random.randrange(n)
    j = random.randrange(n)
    lilManyValues[i,j] = 1
print("Fraction nonzero: " + str(lilManyValues.count_nonzero() / (n*n)))
    
time1 = time()
lilManyValues = lilManyValues * lilManyValues
print("Time for lil multiply: " + str(time() - time1))

denseManyValues = lilManyValues.toarray()
time1 = time()
denseManyValues = denseManyValues * denseManyValues
print("Time for dense multiply: " + str(time() - time1))

Fraction nonzero: 0.09516356
Time for lil multiply: 28.344743967056274
Time for dense multiply: 0.449354887008667


# Compressed Sparse Row (csr_matrix) and Compressed Sparse Column (csc_matrix)

In [41]:
csr1 = lil1.tocsr()
csc1 = lil1.tocsc()
csrIdentity = lilIdentity.tocsr()
cscIdentity = lilIdentity.tocsc()

In [42]:
time1 = time()
csr1 = csr1*csr1
print("Time(s) for csr multiply: " + str(time() - time1))

Time(s) for csr multiply: 0.0008141994476318359


In [43]:
csr2 = csr1
time1 = time()
csr2 = csr2 + csr2
print("Time(s) for csr add: " + str(time() - time1))

Time(s) for csr add: 0.0004820823669433594


In [44]:
iters = 10000
time1 = time()
for i in range(iters):
    index = random.randrange(n)
    row = csr1[index,:]
    nnz = row.count_nonzero()
print("Time(s) for accessing rows: " + str(time() - time1))
for i in range(iters):
    index = random.randrange(n)
    row = csr1[:,index]
    nnz = row.count_nonzero()
print("Time(s) for accessing columns: " + str(time() - time1))


Time(s) for accessing rows: 0.869877815246582
Time(s) for accessing columns: 2.2714409828186035


In [45]:
iters = 10000
time1 = time()
for i in range(iters):
    index = random.randrange(n)
    row = csc1[index,:]
    nnz = row.count_nonzero()
print("Time(s) for accessing rows: " + str(time() - time1))
for i in range(iters):
    index = random.randrange(n)
    row = csc1[:,index]
    nnz = row.count_nonzero()
print("Time(s) for accessing columns: " + str(time() - time1))

Time(s) for accessing rows: 1.4538440704345703
Time(s) for accessing columns: 2.289771795272827


# Hindom Optimization Equation 

F(t+1)=(1/(1+mu)) D^(-1/2) M' D^(-1/2) F(t) + (mu/(1+mu)) Y

Create something that looks like M' of Hindom.

In [58]:
n = 100 
M = lil_matrix((n,n))
numValues = 500
for iter in range(numValues):
    i = random.randrange(n)
    j = random.randrange(n)
    M[i,j] = random.random() # [0,1)
print("Fraction nonzero: " + str(M.count_nonzero() / (n*n)))
M = M.tocsr()

Fraction nonzero: 0.049


Actually have to create an affinity matrix.

In [59]:
import math
W = M.toarray()
for i in range(n):
    for j in range(n):
        if i == j:
            W[i,j] = 0
        else:
            normed = np.linalg.norm(M[i,:].toarray() - M[j,:].toarray(), ord=2)
            W[i,j] = math.exp(-pow(normed,2))
M = W
print(M.shape)

(100, 100)


Create diagonal matrix D

In [60]:
data = np.squeeze(np.asarray(M.sum(axis=1))) # sum the rows
offsets = np.array([0])
D = dia_matrix((data, offsets), shape=(n,n))

Calculate the exponent (-1/2), which is easy for a diagonal.  We can just take each diagonal element and raise it to (-1/2).

In [61]:
D = D.tocsr() # Convert to csr because can't use subscripting
for i in range(n):
    D[i,i] = D[i,i] ** (-1/2) 

Create the matrix S

In [62]:
S = D * M * D

mu is just how we weight each piece, smoothness vs fidelity to known labels

In [71]:
mu = 0.95
alpha = 1/(1 + mu)
beta = mu/(1 + mu)

Create some labels. 

In [72]:
Y = np.zeros((n,2))
# Set some to be malicious and benign
for i in range(int(n/10)):
    index = random.randrange(n)
    if i % 2 == 0:
        Y[index,0] = 1
        Y[index,1] = 0
    else:
        Y[index,0] = 0
        Y[index,1] = 1

Set F to be Y.

In [73]:
F = Y
print(F)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


Do one iteration

In [74]:
F = alpha * S.dot(F) + beta * Y
print(F)

[[0.01819293 0.01902999]
 [0.03733626 0.03545096]
 [0.03258367 0.03093835]
 [0.01099644 0.01766843]
 [0.03199469 0.04121595]
 [0.01773392 0.01627416]
 [0.03777996 0.0366425 ]
 [0.01121374 0.00950491]
 [0.04643648 0.03420793]
 [0.02383316 0.50692371]
 [0.03820814 0.03646696]
 [0.03733517 0.51644383]
 [0.02775491 0.02671696]
 [0.02222738 0.50721453]
 [0.02790699 0.0349064 ]
 [0.03110697 0.02472131]
 [0.02907564 0.02760746]
 [0.03018273 0.03833527]
 [0.02598526 0.02829244]
 [0.52203061 0.04913848]
 [0.01843969 0.01810665]
 [0.49929841 0.01095624]
 [0.04150207 0.03383761]
 [0.02276042 0.02158016]
 [0.03361183 0.03671754]
 [0.04207525 0.03995065]
 [0.03513928 0.02511682]
 [0.03302272 0.02933316]
 [0.01128493 0.01264144]
 [0.02185723 0.0262848 ]
 [0.03813498 0.0371606 ]
 [0.0485999  0.51876266]
 [0.01288624 0.0120815 ]
 [0.0385383  0.03828856]
 [0.03579398 0.03244376]
 [0.02374663 0.03706696]
 [0.04688025 0.04451303]
 [0.51826587 0.03226659]
 [0.01020081 0.01123615]
 [0.02348981 0.02230368]


In [56]:
for i in range(10000):
  F = alpha * S.dot(F) + beta * Y
  
print(F)

[[0.03356588 0.03836037]
 [0.03674686 0.04023443]
 [0.0287061  0.0299738 ]
 [0.02666772 0.02875631]
 [0.02885671 0.03138589]
 [0.03496911 0.03813806]
 [0.03211163 0.03521569]
 [0.03363772 0.03570972]
 [0.02965683 0.03224355]
 [0.03055308 0.03292165]
 [0.02456115 0.02584642]
 [0.02837579 0.03114527]
 [0.03592283 0.03917608]
 [0.02840968 0.02989552]
 [0.02952351 0.03171736]
 [0.03189601 0.03356563]
 [0.02940074 0.03122527]
 [0.02232039 0.02401968]
 [0.03122794 0.03206578]
 [0.03024414 0.03211467]
 [0.02888487 0.03098044]
 [0.03016075 0.03258503]
 [0.03441442 0.03390123]
 [0.02752374 0.03042162]
 [0.35442824 0.02435757]
 [0.02871903 0.03090021]
 [0.3593691  0.02913966]
 [0.03324658 0.03550407]
 [0.03404835 0.03722015]
 [0.02882221 0.03141782]
 [0.35947463 0.02837149]
 [0.01924021 0.02016985]
 [0.03011126 0.03240521]
 [0.02823934 0.03027398]
 [0.03197743 0.03457547]
 [0.02607648 0.02799856]
 [0.03079991 0.03320382]
 [0.02585926 0.02779587]
 [0.03249593 0.03505561]
 [0.03381437 0.03688468]
